## Load Model(s)

In [1]:
import autogen
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file = 'models_list.json',
    filter_dict = {"model": "gpt4-0125preview-128k"}
)

# config_list

## Create the function to be used as a Tool

In [2]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]

def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

## Create the Assistant Agent (associated to its LLM) and the User Agent (no LLM associated with)

In [3]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

## Register Tool
- **assistant agent** --> function *signature* --> LLM
- **user agent** --> function *object* --> LLM

In [4]:
# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

## One alternative to register the tool with both assistant and user agents... in a single step

In [ ]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

In [5]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_lUapiWQ2OuFZCsEMnXBJEoQ9): calculator *****
Arguments: 
{"a": 232, "b": 32, "operator": "-"}
***************************************************************************
***** Suggested tool call (call_7Z6XsbteXTI1xTpmDvg8OGiY): calculator *****
Arguments: 
{"a": 44232, "b": 13312, "operator": "+"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_lUapiWQ2OuFZCsEMnXBJEoQ9) *****
200
**********************************************************************

-------------------------------------

In [7]:
user_proxy.send(recipient= assistant, message="""Please use the arithmetic rules!""")

User (to Assistant):

Please use the arithmetic rules!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

Given the arithmetic principles of order of operations, also known as PEMDAS (Parentheses, Exponents, Multiplication and Division from left to right, Addition and Subtraction from left to right), let's revisit the calculation: \((44232 + 13312 / (232 - 32)) * 5\).

1. **Parentheses first**: Calculate the subtraction inside the parentheses \((232 - 32)\).
2. **Division second**: Divide \(13312\) by the result of step 1.
3. **Addition third**: Add \(44232\) to the result of step 2.
4. **Multiplication last**: Multiply the result of step 3 by \(5\).

The correct steps following these rules are:
- Step 1: Calculate \(232 - 32\).
- Step 2: Divide \(13312\) by the result of Step 1.
- Step 3: Add the result of Step 2 to \(44232\).
- Step 4: Multiply the result of Step 3 by \(5\).

Since we already have some